In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# with open("options.txt", 'r') as f:
#     options = f.readlines()
#     options = {option.split("=")[0]: option.split("=")[1].strip() for option in options}
# print(options)

In [6]:
RUG = pd.read_pickle("rug_timeseries.pkl")
RUG = RUG.filter(['Oosterbeek-hoog\Fletcher BV|Supplied'])
RUG

,Oosterbeek-hoog\Fletcher BV|Supplied
date,
2017-01-01 00:00:00+01:00,1.03
2017-01-01 00:01:00+01:00,1.03
2017-01-01 00:02:00+01:00,1.03
2017-01-01 00:03:00+01:00,1.03
2017-01-01 00:04:00+01:00,1.03
...,...
2018-12-31 23:55:00+01:00,0.68
2018-12-31 23:56:00+01:00,0.68
2018-12-31 23:57:00+01:00,0.68


In [7]:
# fix random seed for reproducibility
# tf.random.set_seed(7)

In [8]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(RUG)

In [9]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

704304 346896


In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [12]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [13]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [15]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=8, batch_size=1, verbose=2)

Epoch 1/8
704302/704302 - 1740s - loss: 9.4332e-05 - accuracy: 1.3489e-04 - 1740s/epoch - 2ms/step
Epoch 2/8
704302/704302 - 1748s - loss: 7.9805e-05 - accuracy: 1.3489e-04 - 1748s/epoch - 2ms/step
Epoch 3/8
704302/704302 - 1798s - loss: 7.8799e-05 - accuracy: 1.3489e-04 - 1798s/epoch - 3ms/step
Epoch 4/8
704302/704302 - 1941s - loss: 7.8390e-05 - accuracy: 1.3489e-04 - 1941s/epoch - 3ms/step
Epoch 5/8
704302/704302 - 2004s - loss: 7.8195e-05 - accuracy: 1.3489e-04 - 2004s/epoch - 3ms/step
Epoch 6/8
704302/704302 - 1973s - loss: 7.7963e-05 - accuracy: 1.3489e-04 - 1973s/epoch - 3ms/step
Epoch 7/8
704302/704302 - 2003s - loss: 7.7805e-05 - accuracy: 1.3489e-04 - 2003s/epoch - 3ms/step
Epoch 8/8
704302/704302 - 2016s - loss: 7.7729e-05 - accuracy: 1.3489e-04 - 2016s/epoch - 3ms/step


In [16]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

10841/10841 [==============================] - 14s 1ms/step
Train Score: 0.06 RMSE
Test Score: 0.06 RMSE
